In [ ]:
import re

lista_all = []

for i in range(1,104):
    
    try:
        i = str(i)
        f = open('mapas/'+i+'.txt', 'r')
        lista_coordenadas = f.readlines()
    
        res =[]
        lista_res = []
        qtd = len(lista_coordenadas)


        for i in lista_coordenadas:     

            if('-' in i):
                #i = i.strip()
                i = " ".join(i.split())
                lista_res.append(i)

        #pega as coordenadas
        res_long = []
        res_lat = []
        for i in lista_res:
            res.append(i.split(" "))

        for i in range(len(res)):
            res[i][0] = float(res[i][0])
            res[i][1] = float(res[i][1])

            res_long.append(res[i][0])
            res_lat.append(res[i][1])

        lista_res = res

        res_lat.sort()
        res_long.sort()

        item = []
        item.append(res_long[0])
        item.append(res_lat[0])
        item.append(res_long[-1])
        item.append(res_lat[-1])
        
        

        lista_all.append(item)
    except:
        print("Arquivo "+i+" não existe")

In [ ]:
for i in range(len(lista_all)):
    print(i+1,":",lista_all[i])

In [ ]:
!pip install tweepy

In [ ]:
import tweepy
import pandas as pd
import datetime
import numpy as np

In [ ]:
bearer_token = 'seu_bearer_token'

api_key = 'seu_api_key'
api_secret = 'seu_api_secret'

access_token = 'seu_access_token'
access_token_secret = 'seu_access_token_secret'

In [ ]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

client = tweepy.Client(auth)

In [ ]:
client = tweepy.Client(bearer_token, api_key, api_secret, access_token, access_token_secret)

In [ ]:
l1 = list(np.arange(1,54))
l2 = [56,57,58,62,63,64,65,66,69,70,73,75,77,78,80,81,85,87,89,90,91,92,96,97,98,99,101,102,103]
areas = l1+l2
len(areas)

In [ ]:
start_time = '2023-06-20T00:00:00Z'
end_time = '2023-06-27T09:00:00Z'
tweets_fields_list = ['created_at','id','text', 'entities', 'in_reply_to_user_id', 'public_metrics', 'geo']
expansions_list = ['author_id', 'geo.place_id']

In [ ]:
cont=0
for i in lista_all:
    
    box = "".join(str(i[0])+" "+str(i[1])+" "+str(i[2])+" "+str(i[3]))
    list_response = []
    
    query = 'bounding_box:['+box+']'
    #query = 'bounding_box:['+box+'] (palavra_chave OR palavra_chave OR palavra-chave etc)'
    for response in tweepy.Paginator(client.search_recent_tweets, query=query, 
                                     tweet_fields=tweets_fields_list, expansions=expansions_list, place_fields=['place_type', 'geo'],
                                     start_time=start_time, end_time = end_time,
                                     max_results=100, limit=1000):
        #places = {p["id"]: p for p in response.includes['places']}
        try:
            places = {p["id"]: p for p in response.includes['places']}
            
        except:
            continue
        
        if response is not None:   
            #print("response not none")
            for tweet in response.data:
                #print("-------------Tweet----------------")
                ##print(tweet.created_at.strftime('%Y-%m-%d'))
                #print(tweet.author_id,tweet.id,tweet.created_at,tweet.text, tweet.public_metrics['reply_count'])
                aux = [tweet.author_id, tweet.id, tweet.created_at.strftime('%Y-%m-%d'), tweet.text]
                list_response.append(aux)
                if tweet.geo:
                    if places[tweet.geo['place_id']]:
                        place = places[tweet.geo['place_id']]
                        
    
    df = pd.DataFrame(list_response, columns=['author_id', 'tweet_id', 'created_at', 'text'])
    df.to_csv('coleta_tweets/area_'+str(areas[cont])+'/tweets_area_'+str(cont)+'_'+start_time+'|'+end_time+'.csv')
    cont+=1